# Dynamic Programming Notes

## Dynamic Programming Foundations
Dynamic programming is simply recursion on steroids, we add efficiency to recursive sub-problems to avoid duplicate computations.

In [ ]:
## Fibonacci Sequence Recursion
def fibonacci(n):
    if n == 1:
        return 1
    if n == 2:
         return 1
    return fibonacci(n-1) + fibonacci(n-2)


fibonacci(10)

55

In [ ]:
## Fibonacci Sequence Top-Down DP
cache = dict() #n is the key
def fibonacci(n):
    ##Base cases
    if n == 1:
        return 1
    if n == 2:
         return 1
    ## Retrieve from cache
    if n in cache.keys():
        return cache[n]

    ## Repetitive sub-problem (Always save to cache first)
    cache[n] = fibonacci(n-1) + fibonacci(n-2)
    return cache[n]


fibonacci(10)

55

In [6]:
## Fibonacci Sequence Bottom-Up DP
def fibonacci(n):
    ##Base cases
    if n == 1:
        return 1
    if n == 2:
         return 1
    
    ##Iterative Solution
    a,b = 1,1
    for i in range(2,n):
        c = a + b
        a = b
        b = c
    return c


fibonacci(10)

55

In summary, let's look at the Big-O notations:
|  | Recursion | Top-Down | Bottom-Up |
|---|---|---|---|
| Runtime | O(2^n) | O(n) | O(n) |
| Space | O(1) | O(n) | O(1) |

## Zero-One Knapsack
Maximise return based on a basket of items. Each item is either in the knapsack or not in the knapsack.

In [ ]:
### Top-Down Dynamic Programming
cache = dict() ## (maxW, weights): whatever the knapsack computes
def knapsack(maxW, weights, profits):
    ##Base Cases
    if len(weights) == 0:
        return 0
    if len(weights) == 1:
        if weights[0] > maxW:
            return 0
        else:
            return profits[0]

    ## retrieve from precomputed responses
    if (maxW,tuple(weights)) in cache.keys():
        return cache[(maxW,tuple(weights))]

    ### Repetitive Subproblem
    if weights[-1] > maxW:
        cache[(maxW,tuple(weights))] = knapsack(maxW, weights[:-1],profits[:-1])
        return cache[(maxW,tuple(weights))]
    else:
        a = knapsack(maxW, weights[:-1],profits[:-1])  #ignore the last item
        b = profits[-1] + knapsack(maxW-weights[-1],weights[:-1],profits[:-1]) #include the last item
        cache[(maxW,tuple(weights))] = max(a,b)
        return cache[(maxW,tuple(weights))]
    

knapsack(18,[1,3,5,7],[2,4,7,10])

In summary, let's look at the Big-O notations (Where `m` is the number of items and `n` is the size of the knapsack):
|  | Recursion | Top-Down | Bottom-Up |
|---|---|---|---|
| Runtime | O(2^n) | O(n*m) | O(n*m) |
| Space | O(1) | O(n*m) | O(n) |

Here are some of the real-world applications of the Zero-One Knapsack:
1. Data Compression: Choosing a subset of files to compress within a storage limit while preserving important data
2. Task Scheduling: Choosing a set of tasks to fit into a schedule while maximising the expected output
3. Investment/Resource Allocation: Choosing which assets to allocate investments based on risks and returns

## Unbounded Knapsack
Maximise return based on a basket of items. Each item can be selected multiple times into the knapsack.

In [11]:
### Top Down DP – Minimum number of coins to give change C
cache = dict() ### key is (coins,amount)

def change(coins, amount):
    ##Base Cases
    if amount == 0:
        return 0
    if len(coins) == 0 and amount > 0:
        return float('inf')
    if len(coins) == 1:
        if amount < coins[0]:
            return float('inf')
        elif amount % coins[0] == 0:
            return amount/coins[0]
        else:
            return float('inf')
    
    ## Retrieve value from cache
    if (tuple(coins),amount) in cache.keys():
        cache[(tuple(coins),amount)]

    ### Repetitive subproblem
    ##The coin value is greater than the change amount
    if coins[0]>amount: 
        cache[(tuple(coins),amount)] = change(coins[1:], amount) #Exclude the coin
        return cache[(tuple(coins),amount)]
    
    ##The coin is not greater than the change amount
    a = 1 + change(coins, amount-coins[0]) #Include the coin
    b = change(coins[1:], amount) #Exclude the coin
    cache[(tuple(coins),amount)] = min(a,b)
    return cache[(tuple(coins),amount)]

def coinchange(coins,amount):
    result = change(coins, amount)
    if result == float('inf'):
        return -1
    return result


coinchange([2],3)

-1

In [13]:
## Bottom-up DP Approach

def coinchange(amount, coins):
    ## Base Cases
    if amount == 0:
        return 0
    if len(coins) == 1: #3 [2], 4 [2]
        if coins[0] > amount:
            return float('inf')
        else:
            return float('inf') if amount%coins[0] != 0 else amount/coins[0]
        
    ## Iterative solution
    prev = [0] + [float('inf')] * amount
    for coin in coins:
        curr = [0] * (amount + 1)
        for i in range(1,amount+1):
            if coin > i:
                curr[i] = prev[i]
            else:
                a = prev[i]
                b = 1 + curr[i-coin]
                curr[i] = min(a,b)
        prev = curr

    return curr[-1]
        

coinchange(11,[1,3,5,7])


3

In summary, let's look at the Big-O notations (Where `m` is the number of coins and `n` is the amount of change):
|  | Recursion | Top-Down | Bottom-Up |
|---|---|---|---|
| Runtime | O(2^n) | O(n*m) | O(n*m) |
| Space | O(1) | O(n*m) | O(n) |

Here are some real world applications for the Unbounded Knapsack:
1. **Investment/Resource Allocation:** Choosing which assets to allocate investments based on risks and returns
2. **Inventory Management:** Deciding how many units of each product to stock in a warehouse to maximize profit while considering storage space limitations.
3. **Production Planning:** Choosing the optimal production quantity of different products to maximize revenue within a production capacity constraint.
4. **Investment Portfolio Optimization:** Selecting the best mix of investments (like stocks and bonds) to maximize potential returns while managing risk within a fixed budget.
5. **Cutting Raw Materials/Dividing a Fixed Resources:** Finding the most efficient way to cut raw materials (like fabric or metal sheets) into smaller pieces to minimize waste. Also applied for division of resources to optimise returns e.g. land splits for sales, planning real estate (e.g. what type of houses to fit in an architectural plan), etc.

## Longest Common Subsequence
Given two strings what is the length of the longest common subsequence.
This DP pattern also applies for the Longest Common Substring/Prefix/Suffix and Longest Palindromic Substring/Subsequence

In [29]:
## Top-Down Dynamic Programming
cache = dict()
def lcs(txt1,txt2):
    ##Base cases
    if len(txt1) == 0 or len(txt2) == 0:
        return 0
    
    ## Retrieve from cache
    if (txt1,txt2) in cache.keys():
        return cache[(txt1,txt2)]
    
    if txt1[0] == txt2[0]:
        cache[(txt1,txt2)] = 1 + lcs(txt1[1:],txt2[1:])
        return cache[(txt1,txt2)]
    
    a = lcs(txt1[1:],txt2)
    b = lcs(txt1,txt2[1:])
    cache[(txt1,txt2)] = max(a,b)
    return cache[(txt1,txt2)]


lcs('abcdefghij', 'ecdgi')

4

In [28]:
## Bottom-up Dynamic Programming
def lcs(txt1,txt2):
    ##Base cases
    if len(txt1) == 0 or len(txt2) == 0:
        return 0
    
    prev = [0] + [0]*len(txt1)
    for char in txt2:
        curr = [0] + [0]*len(txt1)
        for i in range(1,len(curr)):
            if char == txt1[i-1]:
                curr[i] = 1 + prev[i-1]
            else:
                curr[i] = max(curr[i-1],prev[i])
        prev = curr

    return curr[-1]


lcs('abcdefghij', 'ecdgi')

4

In summary, let's look at the Big-O notations (Where `m` is the length of `txt2` and `n` is the length of `txt1`):
|  | Recursion | Top-Down | Bottom-Up |
|---|---|---|---|
| Runtime | O(2^n) | O(n*m) | O(n*m) |
| Space | O(1) | O(n*m) | O(n) |

Here are some application of the LCS DP pattern:
1. **Antivirus Software:** Detecting scripts for commonly known viruses
2. **Git Version Control:** Identifying substrings of code that have changed for every commit to the origin repository
3. **Plagiarism Checker:** Identifying plagiarism in pieces of content by comparing with already produced/published content
4. **Genetic Analysis:** Used to compare DNA sequences to find similarities, which can help scientists understand genetic relationships between different organisms